<a href="https://colab.research.google.com/github/KutuDev/DataCentricAIComp/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Announcement worksheet](https://worksheets.codalab.org/worksheets/0x7a8721f11e61436e93ac8f76da83f0e6)

[Leaderboard](https://https-deeplearning-ai.github.io/data-centric-comp/?utm_source=thebatch&utm_medium=newsletter&utm_campaign=dc-ai-competition&utm_content=dl-ai)

[Data inspection discoveries](https://drive.google.com/file/d/162_c0DDd9UdTN206I9Q7A6TmJg3Fay96/view?usp=sharing)

**Results**
* 1. *Baseline dataset*
        * Final acc: 0.67, Test acc: 0.52
        * Final loss: 0.94, Test loss: 1.38

* 2. *Data v2*
        * Final acc: 0.78, Test acc: 0.69
        * Final loss: 0.93, Test loss: 1.66

**To-Dos**

**Aim:** 0.95+ accuracy 

✅Train the resnet baseline model on their 3k plus data towards obtaining the 0.65 accuracy score. This might require include using their 32 by 32 pixel script for image resizing, deleting corrupted files, and fixing labels.

✅Inspect the 3k+ images for bad labels, bad image, etc. Fix all anomalies found. 

✅Download and read papers that uses the Roman MNIST dataset. We can get links to the dataset and other useful information.  


✅Inspect the image classes distribution and take necessary actions. It's important that the dataset isn't skewed. 

✅Use fastai's image data augmentation functions 

✅Gather more Roman MNIST data in an increase rate of not more than 10x of the 3k data size. This data gathering can be done using any means available, web scraping inclusive.

**N.B:** At the end of each steps above, we retrain the baseline model on the dataset in order to observe the effect on the model accuracy score.

# Notes

* RENAME EACH FILES. Add an auto-increment serial number at the start of each names. This would aid easy lookup of images. Doing this will require creating a doc that contains the previous and new name.
* the noise on some images can be erased using photo editing tools. This tool can also be used for data augmentation such as crop, resize.
* the images labelled as 'trash' in [Data inspection discoveries](https://drive.google.com/file/d/162_c0DDd9UdTN206I9Q7A6TmJg3Fay96/view?usp=sharing) should be moved to a new folder having a new class named 'unrecognized'

* in the I folder, there are images that look like small figure 1
* some images needs to be cropped
* some images have 1/2 vertical noisy lines
* in the II folder, some images written with marker are so small, hence, they appear like roman numeral I.
* in the II folder, some images have a bottom vertical noisy line
* images' stroke are either thick or thin
* some images are small lettered roman numeral 
* some images are capital lettered roman numeral
* some images are capita+bars lettered roman numeral
* some images are dashes
* images are black on white background

In [1]:
!git clone https://github.com/KutuDev/DataCentricAIComp

Cloning into 'DataCentricAIComp'...
remote: Enumerating objects: 45, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 45 (delta 21), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (45/45), done.


In [2]:
def unzipper(in_folderPath, out_folderPath):
    import tarfile

    file = tarfile.open(in_folderPath)
    file.extractall(out_folderPath)
    file.close()


In [3]:
dataset_name = 'data_v2' 

unzipper(in_folderPath=f'/content/DataCentricAIComp/{dataset_name}.tar.gz', out_folderPath='./images')
unzipper(in_folderPath='/content/DataCentricAIComp/label_book.tar.gz', out_folderPath='./images')

In [5]:
# import shutil

# shutil.move(f"/content/images/Users/Samuel/Desktop/{dataset_name}", "/content/images")

In [ ]:
# !pip install pysimplegui

In [ ]:
# !apt-get install -y xvfb # Install X Virtual Frame Buffer
# import os
# os.system('Xvfb :1 -screen 0 1600x1200x16  &')    # create virtual display with size 1600x1200 and 16 bit color. Color can be changed to 24 or 8
# os.environ['DISPLAY']=':1.0'    # tell X clients to use our virtual DISPLAY :1.0

In [ ]:
# !python '/content/DataCentricAIComp/annotgui.py'

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import json
import sys

directory = "/content/images" 
user_data = directory + f"/{dataset_name}"
valid_data = directory + f"/{dataset_name}"
test_data = directory + "/label_book" # this can be the label book, or any other test set you create

### DO NOT MODIFY BELOW THIS LINE, THIS IS THE FIXED MODEL ###
batch_size = 8
tf.random.set_seed(123)


if __name__ == "__main__":
    train = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/train',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    valid = tf.keras.preprocessing.image_dataset_from_directory(
        user_data + '/val',
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=True,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    total_length = ((train.cardinality() + valid.cardinality()) * batch_size).numpy()
    if total_length > 10_000:
        print(f"Dataset size larger than 10,000. Got {total_length} examples")
        sys.exit()

    test = tf.keras.preprocessing.image_dataset_from_directory(
        test_data,
        labels="inferred",
        label_mode="categorical",
        class_names=["i", "ii", "iii", "iv", "v", "vi", "vii", "viii", "ix", "x"],
        shuffle=False,
        seed=123,
        batch_size=batch_size,
        image_size=(32, 32),
    )

    base_model = tf.keras.applications.ResNet50(
        input_shape=(32, 32, 3),
        include_top=False,
        weights=None,
    )
    base_model = tf.keras.Model(
        base_model.inputs, outputs=[base_model.get_layer("conv2_block3_out").output]
    )

    inputs = tf.keras.Input(shape=(32, 32, 3))
    x = tf.keras.applications.resnet.preprocess_input(inputs)
    x = base_model(x)
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dense(10)(x)
    model = tf.keras.Model(inputs, x)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(lr=0.0001),
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=["accuracy"],
    )
    model.summary()
    loss_0, acc_0 = model.evaluate(valid)
    print(f"loss {loss_0}, acc {acc_0}")

    checkpoint = tf.keras.callbacks.ModelCheckpoint(
        "best_model",
        monitor="val_accuracy",
        mode="max",
        save_best_only=True,
        save_weights_only=True,
    )

    history = model.fit(
        train,
        validation_data=valid,
        epochs=100,
        callbacks=[checkpoint],
    )

    model.load_weights("best_model")

    loss, acc = model.evaluate(valid)
    print(f"final loss {loss}, final acc {acc}")

    test_loss, test_acc = model.evaluate(test)
    print(f"test loss {test_loss}, test acc {test_acc}")

In [ ]:
# import os
# import tarfile
# def tardir(path, tar_name):
#     os.chdir(r'C:\Users\Samuel\Desktop')
#     with tarfile.open(tar_name, "w:gz") as tar_handle:
#         for root, dirs, files in os.walk(path):
#             for file in files:
#                 tar_handle.add(os.path.join(root, file))
# tardir('data_v2', 'data_v2.tar.gz')
# tar.close()

In [22]:
y_pred = model.predict(valid)
np.argmax(y_pred[0])

3

In [25]:
# y_pred_labels = [np.argmax(i) for i in y_pred]
print(y_pred_labels)

[3, 8, 9, 6, 0, 6, 7, 2, 9, 8, 7, 6, 2, 6, 9, 6, 2, 0, 7, 4, 8, 3, 2, 7, 2, 4, 7, 2, 7, 4, 4, 9, 7, 0, 8, 2, 4, 7, 2, 5, 9, 7, 2, 2, 8, 1, 1, 5, 2, 6, 5, 3, 3, 6, 2, 9, 4, 1, 2, 6, 1, 1, 0, 4, 3, 0, 7, 4, 1, 3, 8, 8, 9, 1, 6, 8, 9, 7, 9, 1, 4, 4, 0, 7, 7, 6, 0, 9, 7, 6, 9, 9, 1, 0, 5, 8, 7, 7, 8, 2, 2, 6, 7, 9, 0, 3, 6, 3, 3, 5, 2, 0, 8, 2, 0, 6, 8, 2, 4, 6, 8, 4, 9, 6, 1, 6, 4, 5, 2, 5, 6, 5, 2, 7, 0, 3, 7, 9, 2, 9, 8, 0, 3, 0, 8, 5, 3, 6, 4, 3, 7, 9, 3, 6, 6, 7, 2, 8, 6, 9, 5, 2, 8, 1, 7, 0, 0, 6, 3, 2, 9, 2, 9, 8, 2, 3, 0, 7, 4, 5, 2, 6, 2, 0, 7, 7, 4, 8, 7, 2, 9, 6, 4, 8, 6, 1, 2, 0, 0, 2, 6, 4, 2, 2, 1, 3, 7, 7, 8, 0, 5, 1, 0, 0, 2, 3, 0, 9, 6, 1, 4, 8, 4, 5, 9, 8, 5, 0, 0, 8, 6, 7, 7, 8, 1, 8, 7, 7, 9, 5, 7, 4, 7, 3, 4, 8, 4, 6, 3, 1, 2, 8, 1, 7, 6, 4, 3, 9, 0, 2, 8, 1, 8, 8, 3, 1, 6, 4, 5, 9, 7, 4, 1, 4, 7, 4, 2, 1, 3, 9, 6, 9, 6, 3, 6, 8, 1, 7, 2, 5, 0, 6, 0, 7, 4, 6, 9, 9, 2, 6, 8, 0, 3, 8, 0, 2, 9, 8, 6, 3, 8, 4, 8, 8, 6, 7, 6, 8, 7, 9, 2, 1, 8, 6, 3, 3, 5, 4, 1, 0, 4, 2, 2, 

In [65]:
dir(valid)

['_GeneratorState',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_add_variable_with_custom_getter',
 '_apply_options',
 '_as_serialized_graph',
 '_batch_size',
 '_checkpoint_dependencies',
 '_consumers',
 '_deferred_dependencies',
 '_drop_remainder',
 '_flat_shapes',
 '_flat_structure',
 '_flat_types',
 '_functions',
 '_gather_saveables_for_checkpoint',
 '_graph',
 '_graph_attr',
 '_handle_deferred_dependencies',
 '_has_captured_ref',
 '_input_dataset',
 '_inputs',
 '_list_extra_dependencies_for_serialization',
 '_list_functions_for_serialization',
 '_lookup_de

In [74]:
# a = [i for i in valid.enumerate().as_numpy_iterator()]
# b = np.asarray(a).astype(np.float32)
# c = tf.convert_to_tensor(b)

In [73]:
# tf.math.confusion_matrix(labels=b,predictions=y_pred_labels)
# dir(valid)

In [75]:
# valid.take(0)

In [ ]:
# Predict the values from the validation dataset
y_pred = model.predict(valid)
# Convert predictions classes to one hot vectors 
y_pred_classes = np.argmax(y_pred, axis = 1) 

y_test = 

y_test = np.argmax(y_test, axis=1)

# Errors are difference between predicted labels and true labels
errors = (y_pred_classes - y_test != 0)

y_pred_classes_errors = y_pred_classes[errors]
y_pred_errors = y_pred[errors]
y_true_errors = y_test[errors]
x_test_errors = x_test[errors]

# Probabilities of the wrong predicted numbers
y_pred_errors_prob = np.max(y_pred_errors, axis = 1)

# Predicted probabilities of the true values in the error set
true_prob_errors = np.diagonal(np.take(y_pred_errors, y_true_errors, axis=1))

# Difference between the probability of the predicted label and the true label
delta_pred_true_errors = y_pred_errors_prob - true_prob_errors

# Sorted list of the delta prob errors
sorted_dela_errors = np.argsort(delta_pred_true_errors)

In [ ]:
import matplotlib.pyplot as plt

def display_errors(errors_index,img_errors,pred_errors, obs_errors):
    """ This function shows 6 images with their predicted and real labels"""
    n = 0
    nrows = 2
    ncols = 3
    fig, ax = plt.subplots(nrows,ncols,sharex=True,sharey=True)
    for row in range(nrows):
        for col in range(ncols):
            error = errors_index[n]
            ax[row,col].imshow((img_errors[error]).reshape((32,32,3)))
            ax[row,col].set_title("Predicted label :{}\nTrue label :{}".format(pred_errors[error],obs_errors[error]))
            n += 1

# Show the top 6 errors
most_important_errors = sorted_dela_errors[-6:]
display_errors(most_important_errors, x_test_errors, y_pred_classes_errors, y_true_errors)